# Projet de Détection et de Tracking Multi-Caméras (MTH_2329)

Ce projet implémente un système complet de tracking d'objets à travers un réseau de caméras à la DRSI.

In [ ]:
import os
import pandas as pd
from config import CAMERAS
from tracking_logic import ObjectTrackingSystem
from IPython.display import display

# Initialisation du système avec YOLOv8
tracker_system = ObjectTrackingSystem(model_weight='yolov8n.pt')

## 1. Dictionnaire des Caméras et Localisations
Configuration des caméras (Mappage entre fichiers et positions physiques).

In [ ]:
df_config = pd.DataFrame.from_dict(CAMERAS, orient='index')
display(df_config[['location', 'path']])

## 2. Exécution du Tracking sur les vidéos
Cette cellule traite les vidéos. Si une vidéo est déjà présente dans `output_tracked`, elle sera sautée. 

> **Note** : Si vous voyez '0 objets', c'est peut-être parce que les fichiers de données (.csv) sont manquants dans `output_tracked`. Dans ce cas, supprimez les vidéos dans `output_tracked` et relancez cette cellule.

In [ ]:
output_dir = 'output_tracked'
processed_count = 0

for cam_name, info in CAMERAS.items():
    output_video = os.path.join(output_dir, f"{cam_name}_tracked.mp4")
    output_csv = os.path.join(output_dir, f"{cam_name}_results.csv")
    
    if not os.path.exists(info['path']):
        continue
        
    # On ne saute que si la vidéo ET le CSV existent
    if os.path.exists(output_video) and os.path.exists(output_csv):
        print(f"Déjà traité : {cam_name} (skip)")
        continue
        
    print(f"Traitement en cours : {cam_name}...")
    tracker_system.process_video(
        camera_name=cam_name, 
        video_path=info['path'], 
        zone_poly=info['zone_alerte'],
        output_folder=output_dir
    )
    processed_count += 1

print(f"Traitement terminé pour {processed_count} caméras.")

## 3. Analyse des trajectoires (Single Target Tracking)
Nous rechargeons les données depuis les fichiers CSV pour l'analyse globale.

In [ ]:
tracker_system.load_existing_results(output_dir)
df_full = tracker_system.get_dataframe()

if not df_full.empty:
    print(f"Nombre total de détections : {len(df_full)}")
    df_summary = tracker_system.analyze_trajectories()
    print("Résumé des trajectoires détectées :")
    display(df_summary)
else:
    print("Erreur : Aucune donnée de détection trouvée. Vérifiez que les fichiers .csv existent dans output_tracked.")

## 4. Détails des Objets (BBox, Centre, Temps, etc.)
Affichage des informations détaillées par frame pour chaque objet.

In [ ]:
if not df_full.empty:
    # On affiche les 10 premières lignes pour voir la structure (BBox, x_c, y_c, etc.)
    display(df_full[['camera', 'frame', 'timestamp', 'obj_id', 'class', 'bbox', 'x_c', 'y_c', 'w', 'h']].head(10))
else:
    print("Pas de données à afficher.")

## 5. Parcours Multi-Caméra et Suivi Global
Cette section permet de voir l'historique d'un objet d'une caméra à l'autre.

In [ ]:
def global_tracking_report(target_class='person'):
    df_summary = tracker_system.analyze_trajectories()
    if df_summary.empty:
        return "Aucune donnée disponible."
        
    path = df_summary[df_summary['class'] == target_class].sort_values(by='start_time')
    if path.empty:
        return f"Aucun objet de classe {target_class} n'a été trouvé."
    
    print(f"--- Parcours de l'objet '{target_class}' ---")
    for i, row in path.iterrows():
        print(f"Vu dans {row['camera']} | Entrée: {row['entry_point']} | Direction: {row['direction']} | État final: {row['state']}")
    
    last_cam = path.iloc[-1]['camera']
    print(f"\nPoint de sortie final : Caméra {last_cam} à la position {path.iloc[-1]['exit_point']}")
    return path

display(global_tracking_report())